Introdução
Esse trabalho é uma analise de duas tabelas que contem dados de filmes, extraídas do IMDB, trabalharei com os dados de filme, os datasets usados serão um dataset com informações básicas de filme e um dataset com notas dos mesmos.

O objetivo é responder as seguintes perguntas: 

1.	A nota média de filmes varia ao longo das décadas?"
2.	Qual a nota média dos filmes por década?
3.	Quais gêneros mais se destacaram em cada década?
4.	O número de filmes lançados aumentou com o tempo?



In [0]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, split, explode
from pyspark.sql.functions import when
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType, StringType, BooleanType
from pyspark.sql.functions import explode, split
from pyspark.sql.functions import col, explode, split, floor, count, avg, desc



In [0]:
# Comando para procurar o caminho dos arquivos
display(dbutils.fs.ls('dbfs:/FileStore/tables'))



path,name,size,modificationTime
dbfs:/FileStore/tables/title_basics.tsv,title_basics.tsv,1000385777,1744401970000
dbfs:/FileStore/tables/title_ratings-1.tsv,title_ratings-1.tsv,27091117,1744401755000
dbfs:/FileStore/tables/title_ratings-2.tsv,title_ratings-2.tsv,27091117,1744402249000
dbfs:/FileStore/tables/title_ratings.tsv,title_ratings.tsv,27091117,1744397172000


In [0]:
 # Carregar os arquivos TSV
 df_basics = spark.read.option("header", True)\
    .option("sep", "\t")\
    .option("inferSchema", True)\
    .csv("dbfs:/FileStore/tables/title_basics.tsv")

    # Visualizar esquema dos dados
print("Esquema de title.basics:")
df_basics.printSchema()



Esquema de title.basics:
root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [0]:
# Carregar os arquivos TSV
df_ratings = spark.read.option("header", True)\
    .option("sep", "\t")\
    .option("inferSchema", True)\
    .csv("dbfs:/FileStore/tables/title_ratings.tsv")

     # Visualizar esquema dos dados
print("Esquema de title.ratings:")
df_ratings.printSchema()


Esquema de title.ratings:
root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



In [0]:
%sql CREATE DATABASE bronze;

In [0]:
df_basics.write.format("delta").mode("overwrite").saveAsTable("bronze.title_basics")
df_ratings.write.format("delta").mode("overwrite").saveAsTable("bronze.title_ratings")

In [0]:
%sql SELECT * FROM bronze.title_basics LIMIT 10

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short
tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [0]:
%sql SELECT * FROM bronze.title_ratings LIMIT 10

tconst,averageRating,numVotes
tt0000001,5.7,2148
tt0000002,5.5,292
tt0000003,6.5,2182
tt0000004,5.3,188
tt0000005,6.2,2926
tt0000006,5.0,211
tt0000007,5.3,907
tt0000008,5.4,2293
tt0000009,5.3,224
tt0000010,6.8,7928


In [0]:
%sql DROP DATABASE silver CASCADE

In [0]:
%sql CREATE DATABASE silver

In [0]:
df_basics.write.mode("overwrite").saveAsTable("silver.title_basics")

In [0]:
# Definir as colunas e seus novos tipos
# (Ajuste conforme as colunas do seu arquivo)
df_converted = df_basics \
    .withColumn("tconst", col("tconst").cast(StringType())) \
    .withColumn("titleType", col("titleType").cast(StringType())) \
    .withColumn("primaryTitle", col("primaryTitle").cast(StringType())) \
    .withColumn("originalTitle", col("originalTitle").cast(StringType())) \
    .withColumn("isAdult",
        when(col("isAdult") == "\\N", None)
        .otherwise(col("isAdult").cast(IntegerType()))
    ) \
    .withColumn("startYear",
        when(col("startYear") == "\\N", None)
        .otherwise(col("startYear").cast(IntegerType()))
    ) \
    .withColumn("endYear",
        when(col("endYear") == "\\N", None)
        .otherwise(col("endYear").cast(IntegerType()))
    ) \
    .withColumn("runtimeMinutes",
        when(col("runtimeMinutes") == "\\N", None)
        .otherwise(col("runtimeMinutes").cast(IntegerType()))
    ) \
    .withColumn("genres", col("genres").cast(StringType()))

# Verificar esquema resultante
print("Esquema após conversão:")
df_converted.printSchema()

# Mostrar amostra dos dados
display(df_converted.limit(5))

Esquema após conversão:
root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: integer (nullable = true)
 |-- runtimeMinutes: integer (nullable = true)
 |-- genres: string (nullable = true)



tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,short,Carmencita,Carmencita,0,1894,null,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,null,5,"Animation,Short"
tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,null,5,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,null,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,null,1,Short


Verifiquei que algumas colunas estavam com a classsifficação errada, por isso fiz a alteraçao dos tipos.

In [0]:
%sql SELECT tconst, titleType, originalTitle, startYear as year, genres,
CASE 
  WHEN isAdult = 1 THEN TRUE
  WHEN isAdult = 0 THEN FALSE
END AS isAdult
FROM silver.title_basics
WHERE isAdult = 0 OR isAdult = 1
LIMIT 10

tconst,titleType,originalTitle,year,genres,isAdult
tt0000001,short,Carmencita,1894,"Documentary,Short",false
tt0000002,short,Le clown et ses chiens,1892,"Animation,Short",false
tt0000003,short,Pauvre Pierrot,1892,"Animation,Comedy,Romance",false
tt0000004,short,Un bon bock,1892,"Animation,Short",false
tt0000005,short,Blacksmith Scene,1893,Short,false
tt0000006,short,Chinese Opium Den,1894,Short,false
tt0000007,short,Corbett and Courtney Before the Kinetograph,1894,"Short,Sport",false
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,1894,"Documentary,Short",false
tt0000009,movie,Miss Jerry,1894,Romance,false
tt0000010,short,La sortie de l'usine Lumière à Lyon,1895,"Documentary,Short",false


A coluna isAdult se encaixava no padrao do tipo booleano, entao realizei essa transformação.

In [0]:
%sql
CREATE OR REPLACE TABLE silver.title_basics_clean AS
SELECT tconst, titleType, originalTitle, startYear as year, genres,
CASE 
  WHEN isAdult = 1 THEN TRUE
  WHEN isAdult = 0 THEN FALSE
END AS isAdult
FROM silver.title_basics
WHERE isAdult = 0 OR isAdult = 1

num_affected_rows,num_inserted_rows


Apos a transformação para booleano percebi que alguns valores eram diferentes de 0 ou 1, considerei como inconsistencia da tabela, então nessa etapa atualizei a tabela excluindo esses dados.

In [0]:
%sql select * from silver.title_basics_clean
limit 10

tconst,titleType,originalTitle,year,genres,isAdult
tt0000001,short,Carmencita,1894,"Documentary,Short",false
tt0000002,short,Le clown et ses chiens,1892,"Animation,Short",false
tt0000003,short,Pauvre Pierrot,1892,"Animation,Comedy,Romance",false
tt0000004,short,Un bon bock,1892,"Animation,Short",false
tt0000005,short,Blacksmith Scene,1893,Short,false
tt0000006,short,Chinese Opium Den,1894,Short,false
tt0000007,short,Corbett and Courtney Before the Kinetograph,1894,"Short,Sport",false
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,1894,"Documentary,Short",false
tt0000009,movie,Miss Jerry,1894,Romance,false
tt0000010,short,La sortie de l'usine Lumière à Lyon,1895,"Documentary,Short",false


In [0]:
%sql SELECT isAdult, COUNT(*)
FROM silver.title_basics_clean
GROUP BY isAdult 


isAdult,count(1)
true,374472
false,11207116


Fiz a verificação se de fato so estava considerando as instancias com valor de 0 ou 1.


In [0]:
%sql DROP DATABASE gold CASCADE

In [0]:
%sql CREATE DATABASE gold

In [0]:
%sql  CREATE TABLE gold.films_rating
SELECT 
    b.*, 
    r.averageRating, 
    r.numVotes
FROM 
    silver.title_basics_clean as b
INNER JOIN 
    bronze.title_ratings as r
ON 
    b.tconst = r.tconst

num_affected_rows,num_inserted_rows


O modelo de dados escolhido foi a junção das duas tabelas criando uma tabela flat.



In [0]:
%sql
CREATE OR REPLACE TABLE gold.films_rating_with_decade AS
SELECT 
  *, 
  FLOOR(year / 10) * 10 AS decade
FROM gold.films_rating;

num_affected_rows,num_inserted_rows


In [0]:
%sql

SELECT 
  * 
FROM gold.films_rating_with_decade
limit 10;

tconst,titleType,originalTitle,year,genres,isAdult,averageRating,numVotes,decade
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,1894,"Documentary,Short",false,5.4,2293,1890
tt0000015,short,Autour d'une cabine,1894,"Animation,Comedy,Short",false,6.1,1272,1890
tt0000019,short,The Clown Barber,1898,"Comedy,Short",false,5.2,35,1890
tt0000051,short,Campement de bohémiens,1896,"Documentary,Short",false,3.5,39,1890
tt0000058,short,Cortège de tzar au Bois de Boulogne,1896,"Documentary,Short",false,4.4,37,1890
tt0000065,short,Dessinateur: Reine Victoria,1896,Short,false,3.5,33,1890
tt0000095,short,Le papier protée,1896,Short,false,4.8,38,1890
tt0000108,short,Rip Leaving Sleepy Hollow,1896,"Drama,Short",false,4.4,588,1890
tt0000142,short,Buffalo Bill and Escort,1897,"News,Short",false,5.0,34,1890
tt0000145,short,En classe,1897,"Comedy,Short",false,3.9,16,1890


Foi criada a coluna de decada para apoiar as analises e responder as perguntas propostas no projeto.

## Catago de dados.

 Tabela: gold.films_rating_with_decade

Junção entre silver.title_basics_clean e bronze.title_ratings.

| Coluna         | Tipo     | Descrição                                 |
|----------------|----------|-------------------------------------------|
| tconst         | String   | ID do título                              |
| titleType      | String   | Tipo de título                            |
| originalTitle  | String   | Título original                           |
| year           | Integer  | Ano de lançamento                         |
| genres         | String   | Gêneros                                   |
| isAdult        | Boolean  | Conteúdo adulto                           |
| averageRating  | Double   | Avaliação média                           |
| numVotes       | Integer  | Número de votos                           |
| decade         | Integer  | Década de lançamento (ex: 1990, 2000)     |



In [0]:
%sql
SELECT 
  * 
FROM gold.films_rating_with_decade
  where decade  is null
LIMIT 10;

tconst,titleType,originalTitle,year,genres,isAdult,averageRating,numVotes,decade
tt0219466,movie,90° South,\N,Documentary,false,7.4,206,null
tt10409788,tvSeries,Power Panda Posse,\N,Animation,false,8.0,7,null
tt10951260,tvEpisode,"Prequels, Sequels, and How the New ""Dark Crystal"" Fits In",\N,"News,Short",false,6.8,14,null
tt12152386,tvEpisode,Lost and found,\N,"Drama,Sci-Fi,Thriller",false,9.6,7,null
tt12333526,tvMovie,Crime dans l'Hérault,\N,"Crime,Thriller",false,6.4,75,null
tt13106696,movie,Quel che conta è il pensiero,\N,"Comedy,Drama",false,6.6,12,null
tt13226480,tvSeries,WorkInProgress: A Comedy Web-Series,\N,Comedy,false,8.5,6,null
tt14301868,tvEpisode,From Largo Theatre Howie Mandel,\N,"Comedy,Music,Talk-Show",false,8.4,10,null
tt19357774,tvEpisode,Ray Romano/Brooklyn Decker/Joy Downer,\N,"Comedy,Talk-Show",false,6.8,8,null
tt19853122,tvEpisode,Norman Reedus/Paula Pell/Michael 'Beeple' Winkelmann/The Linda Lindas,\N,"Comedy,Music,Talk-Show",false,6.8,14,null


Após a criação observei que havia anos sem preenchimento, então tratei novamente a tabela deixando de fora esses registros, o que impactaria as analises pretendidas.



In [0]:
%sql
SELECT 
  decade,
  AVG(averageRating) AS avg_rating,
  COUNT(*) AS num_movies
FROM gold.films_rating_with_decade 
WHERE decade is not null
GROUP BY decade
ORDER BY decade

decade,avg_rating,num_movies
1870,5.3687499999999995,32
1880,4.947500000000001,80
1890,4.60161472024033,2663
1900,5.07051282051282,3276
1910,5.722336668628608,6796
1920,6.008565279770443,6970
1930,6.038790391764371,13988
1940,6.176289816061014,13374
1950,6.708395030345397,31471
1960,6.824401753864444,53824


Databricks visualization. Run in Databricks to view.

1.	A nota média de filmes varia ao longo das décadas?

R: Sim, podemos  observar que a média de notas foi crescendo com o passar do tempo.
Alguns dos motivos se da pelo avanço tecnologico, a popularização e disseminação do  cinema no mundo, alem do aumento na produção de filmes e o acesso do publico.

In [0]:
%sql
SELECT 
  decade,
  AVG(averageRating) AS avg_rating,
  COUNT(*) AS num_movies
FROM gold.films_rating_with_decade 
WHERE decade is not null
GROUP BY decade
ORDER BY decade

decade,avg_rating,num_movies
1870,5.3687499999999995,32
1880,4.947500000000001,80
1890,4.60161472024033,2663
1900,5.07051282051282,3276
1910,5.722336668628608,6796
1920,6.008565279770443,6970
1930,6.038790391764371,13988
1940,6.176289816061014,13374
1950,6.708395030345397,31471
1960,6.824401753864444,53824


A tabela mostra o resultado para a segunda pergunta de nota media por decada.


In [0]:
# Explodindo a coluna 'genres' 
 # Lê a tabela do catálogo
df = spark.table("gold.films_rating_with_decade")

# Explode os gêneros
df_genres_exploded = df.withColumn(
    "genre", 
    explode(split(col("genres"), ","))
)

# Mostra os resultados
df_genres_exploded.show(5)

+---------+---------+-----------------+----+-----------------+-------+-------------+--------+------+-----------+
|   tconst|titleType|    originalTitle|year|           genres|isAdult|averageRating|numVotes|decade|      genre|
+---------+---------+-----------------+----+-----------------+-------+-------------+--------+------+-----------+
|tt0000014|    short|L'arroseur arrosé|1895|     Comedy,Short|  false|          7.1|    6142|  1890|     Comedy|
|tt0000014|    short|L'arroseur arrosé|1895|     Comedy,Short|  false|          7.1|    6142|  1890|      Short|
|tt0000023|    short|  Baignade en mer|1895|Documentary,Short|  false|          5.7|    1585|  1890|Documentary|
|tt0000023|    short|  Baignade en mer|1895|Documentary,Short|  false|          5.7|    1585|  1890|      Short|
|tt0000044|    short|       Le bivouac|1896|            Short|  false|          4.1|      57|  1890|      Short|
+---------+---------+-----------------+----+-----------------+-------+-------------+--------+---

In [0]:
df_genres_exploded.write.mode("overwrite").saveAsTable("gold.films_genres_exploded")



In [0]:
%sql SELECT 
  decade,
  genre,
  genre_count.total_films,
  RANK() OVER (PARTITION BY decade ORDER BY total_films DESC) AS genre_rank
FROM (
  SELECT 
    decade,
    genre,
    COUNT(*) AS total_films
  FROM gold.films_genres_exploded
  WHERE genre IS NOT NULL AND genre != '\\N'
  GROUP BY decade, genre
) AS genre_count
WHERE genre IS NOT NULL
AND decade IS NOT NULL
QUALIFY genre_rank = 1
ORDER BY decade;

decade,genre,total_films,genre_rank
1870,Short,32,1
1880,Short,79,1
1890,Short,2642,1
1900,Short,3111,1
1910,Short,3700,1
1920,Comedy,2587,1
1930,Comedy,5389,1
1940,Drama,5163,1
1950,Drama,12859,1
1960,Drama,20424,1


In [0]:
%sql SELECT 
  decade,
  genre,
  COUNT(*) AS total_films
FROM 
  gold.films_genres_exploded
WHERE 
  genre IS NOT NULL 
  AND genre != '\\N'  
  AND decade IS NOT NULL-- Garantir que o gênero não seja nulo ou inválido

GROUP BY 
  decade, genre
ORDER BY 
  decade, total_films DESC
LIMIT 10;


decade,genre,total_films
1870,Short,32
1870,Animation,30
1870,Documentary,2
1870,Sport,1
1870,History,1
1880,Short,79
1880,Documentary,24
1880,Animation,2
1880,Sport,1
1880,Drama,1


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
  decade,
  COUNT(*) AS total_films                 
FROM 
  gold.films_genres_exploded           
WHERE 
  decade IS NOT NULL                 
GROUP BY 
  decade                                 
ORDER BY 
  decade;

decade,total_films
1870,66
1880,109
1890,4897
1900,6282
1910,12257
1920,13200
1930,28262
1940,27160
1950,58088
1960,104922


Databricks visualization. Run in Databricks to view.

In [0]:
%sql  WITH movies_by_decade AS (
  SELECT 
    decade,
    COUNT(*) AS movie_count,
    LAG(COUNT(*), 1) OVER (ORDER BY decade) AS prev_decade_count
  FROM gold.films_rating_with_decade
  WHERE decade IS NOT NULL
  GROUP BY decade
)

SELECT 
  decade,
  movie_count,
  prev_decade_count,
  ROUND((movie_count - prev_decade_count) / prev_decade_count * 100, 2) AS growth_percent,
  CASE 
    WHEN decade BETWEEN 1870 AND 1890 THEN 'Era Pioneira'
    WHEN decade BETWEEN 1900 AND 1920 THEN 'Cinema Mudo'
    WHEN decade BETWEEN 1930 AND 1950 THEN 'Era de Ouro'
    WHEN decade BETWEEN 1960 AND 1980 THEN 'Expansão Global'
    WHEN decade >= 1990 THEN 'Era Digital'
  END AS era_cinematografica
FROM movies_by_decade

ORDER BY decade;

decade,movie_count,prev_decade_count,growth_percent,era_cinematografica
1870,32,null,null,Era Pioneira
1880,80,32,150.0,Era Pioneira
1890,2663,80,3228.75,Era Pioneira
1900,3276,2663,23.02,Cinema Mudo
1910,6796,3276,107.45,Cinema Mudo
1920,6970,6796,2.56,Cinema Mudo
1930,13988,6970,100.69,Era de Ouro
1940,13374,13988,-4.39,Era de Ouro
1950,31471,13374,135.31,Era de Ouro
1960,53824,31471,71.03,Expansão Global


Principais Conclusões:
Crescimento Exponencial:

- A produção cinematográfica aumentou em quase todas as décadas, exceto na década atual (incompleta)

- Marcos Históricos:

- 1890-1920:  (popularização do cinema)

- 1940s:  2ª Guerra Mundial

- 2000s: Pico absoluto de produção 

- Tendência Recente:

- A década de 2020 mostra redução (dados parciais), possivelmente devido:

- Pandemia de COVID-19

- Mudança para streaming (menos registros no IMDB)

- Década ainda não concluída

## Autoavaliação

Esse projeto foi desafiador para mim, não tinha conhecimento da ferramenta, porem foi enriquecedor tive problemas com a minha gestão de tempo devido ao trabalho, mas me dediquei o máximo com o tempo que tinha disponível, acredito que poderia ter entregado um trabalho melhor se tivesse mais tempo pois em algumas partes a falta de conhecimento me impediu de fazer um código melhor, mais limpo sem redundâncias. Porém consegui responder as perguntas as quais meu projeto foi pautado. Como melhoria revisaria o código para o tornar mais limpo, implementaria um agendamento para atualização de novos dados, bem como a criação de dashboards para uma visualização das análises de mais fácil compreensão.

Agradeço a oportunidade e o aprendizado.

Tereza Alimos
